---
execute:
  cache: false
  eval: true
  echo: true
  warning: false
jupyter: python3
---

# Multicollinearity and Principle Component Analysis (PCA)

In [ ]:
#| echo: false
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from factor_analyzer import FactorAnalyzer, calculate_kmo
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from spotpython.surrogate.kriging import Kriging
from spotpython.utils.pca import pca_analysis
from spotpython.utils.stats import compute_coefficients_table, condition_index
import xgboost as xgb

First, the data is preprocessed to ensure that it does not contain any NaN or infinite values. Then, PCA is performed to reduce the dimensions and extract the principal components. Finally, a linear regression is conducted on the extracted factors.

::: {#def-multicoll}
#### Multicollinearity and Multicorrelation

Multicorrelation is a general term that describes correlation between multiple variables. Multicollinearity is a specific problem in regression models caused by strong correlations between independent variables, making model interpretation difficult.
:::

## Loading and Preprocessing the Data

We load the data set and display the first few rows. The data set contains information about car sales, including various features such as price, engine size, horsepower, and more.

In [ ]:
df = pd.read_csv("data/car_sales.csv", encoding="utf-8")
print(df.shape)
df.head()

In [ ]:
# Remove the first column (assuming it is an index or non-informative)
df = df.drop(df.columns[0], axis=1)
# print the column names
print(df.columns)

### The Target Variable `y`

Transform sales to log scale; handle negative or zero sales separately.
Explicitly check and handle missing and infinite values.

In [ ]:
df['ln_sales'] = np.log(df['sales'].replace(0, np.nan))
if df['ln_sales'].isnull().any() or np.isinf(df['ln_sales']).any():
    df['ln_sales'] = df['ln_sales'].fillna(df['ln_sales'].median())  # Or any other strategy
y = df['ln_sales']

### The Features `X`

### Numerical Features

### Data Preprocessing

The following steps are performed during data preprocessing:
1. Check for NaN or infinite values in `X`.
2. Replace NaN and infinite values with the median of the respective column.
3. Remove constant or nearly constant columns.
4. Standardize the numerical predictors in `X` using `StandardScaler`.
5. Verify that `X_scaled` does not contain any NaN or infinite values.


In [ ]:
# Use columns from 'price' to 'mpg' as predictors
independent_var_columns = ['price', 'engine_s', 'horsepow', 'wheelbas', 
                           'width', 'length', 'curb_wgt', 'fuel_cap', 'mpg']

# Select those columns, ensuring they are numeric
X = df[independent_var_columns].apply(pd.to_numeric, errors='coerce')

# Handle missing/nans in features by using an appropriate imputation strategy
X = X.fillna(X.median())  # Impute with median or any other appropriate strategy
# Display the first few rows of the features
X.head()

In [ ]:
if X.isnull().any().any():
    print("NaNs detected in X. Filling with column medians.")
    X = X.fillna(X.median())

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
if np.isnan(X_scaled).any() or np.isinf(X_scaled).any():
    raise ValueError("X_scaled contains NaN or infinite values after preprocessing.")
# Convert the scaled data back to a DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
# Display the first few rows of the scaled features
X_scaled.head()

### Categorical Features

In [ ]:
categorical_cols = ['type']  # Replace if more categorical variables exist
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_categorical_encoded = encoder.fit_transform(df[categorical_cols])
# Convert encoded data into a DataFrame
X_categorical_encoded_df = pd.DataFrame(X_categorical_encoded,
                                        columns=encoder.get_feature_names_out(categorical_cols))
X_categorical_encoded_df.describe(include='all')

### Combine non-categorical and categorical (encoded) data

In [ ]:
X_encoded = pd.concat([X_scaled, X_categorical_encoded_df], axis=1)
X_encoded.describe(include='all')

## Fit the Linear Regression Model using statsmodels

In [ ]:
X_encoded_with_const = sm.add_constant(X_encoded)  # Adds a constant term (intercept) to the model
model = sm.OLS(df['ln_sales'], X_encoded_with_const).fit()

## Model Summary and Interpretation

In [ ]:
print(model.summary())

### The OLS Regression Results

The ANOVA table shows a significant F-value (observed in the output as `Prob (F-statistic)`, with a value close to zero). This indicates that using the model is better than estimating the mean. Overall, the regression performs well in modeling the selected target variable (`ln_sales`). Nearly 40% of the variation in the `ln_sales` values is explained by the model, as shown by `Adj. R-squared`.

Although the model fit appears positive, the coefficients (`coef`) reveal that too many predictors are included in the model. There are several non-significant coefficients (`P>|t|` much larger than $0.05$), indicating that these variables contribute little to the model.

### The Coefficient Table

In [ ]:
coeffs_table = compute_coefficients_table(
    model=model,
    X_encoded=X_encoded_with_const,
    y=y,
    vif_table=None
)
print("\nCoefficients Table:")
print(coeffs_table)

The coefficient table further suggests that there may be a problem with multicollinearity. For most predictors, the values of the partial correlations (`Partial r`) decrease significantly compared to the zero-order correlation (`Zero-Order r`).

The tolerance (`Tolerance`) indicates the percentage of variance in a specific predictor that cannot be explained by the other predictors. The low tolerances therefore show that approximately 70%-90%, i.e., 1 minus the `Tolerance` value in percent, of the variance in a given predictor can be explained by the other predictors. When tolerances are close to 0, high multicollinearity exists.

A variance inflation factor (`VIF`) greater than 2 is typically considered problematic, and the smallest VIF in the table is already greater than 2.


## Eigenvalues

Eigenvalues indicate how many factors can be meaningfully extracted.
An eigenvalue greater than 1 suggests that the factor explains more variance than a single variable.

In [ ]:
fa_temp = FactorAnalyzer(n_factors=X_encoded.shape[1], method="principal", rotation=None)
try:
    fa_temp.fit(X_encoded)
    ev, _ = fa_temp.get_eigenvalues()
    # sort eigenvalues in ascending order
    ev = np.sort(ev)
    print("Eigenvalues for each component:\n", ev)
except Exception as e:
    print(f"Error during factor analysis fitting: {e}")
    print("Consider reducing multicollinearity or removing problematic features.")

## Collinearity Diagnostics Based on Eigenvalues

In [ ]:
X_cond = copy.deepcopy(X_encoded)
condition_index_df = condition_index(X_cond)
print("\nCondition Index:")
print(condition_index_df)

The eigenvalue-based collinearity diagnostics confirm that there are serious issues with multicollinearity. Several eigenvalues (`Eigenvalue`) are close to 0, indicating that the predictors are highly correlated and that small changes in the data values can lead to large changes in the coefficient estimates.

The condition indices (`Condition Index`) are calculated as the square roots of the ratios of the largest eigenvalue to each subsequent eigenvalue. Values greater than 15 indicate a potential collinearity problem, while values greater than 30 suggest a severe issue. Five of these indices exceed 30, highlighting a very serious problem with multicollinearity.


### Kayser-Meyer-Olkin (KMO) Measure

The KMO measure (Kaiser-Meyer-Olkin) is a metric for assessing the suitability of data for factor analysis. A KMO value of 0.6 or higher is often considered acceptable; see also [Wikipedia](https://en.wikipedia.org/wiki/Kaiser%E2%80%93Meyer%E2%80%93Olkin). A KMO value below 0.5 indicates that the data is not suitable for factor analysis.

* The KMO measure is based on the correlation and partial correlation between variables.
* It is calculated as follows: For each variable, the ratio of the squared sums of correlations to the squared sums of partial correlations is computed. The formula for the KMO measure is:

$$
KMO = \frac{\displaystyle \underset{j\neq k}{\sum\sum} r_{jk}^2}{\displaystyle \underset{j\neq k}{\sum\sum} r_{jk}^2+\underset{j\neq k}{\sum\sum} p_{jk}^2},
$$

where $r_{ij}^2$ are the squares of the correlations between variables $i$ and $j$, and $p_{ij}^2$ are the squares of the partial correlations between variables $i$ and $j$.

* The KMO measure ranges between 0 and 1.
* Values close to 1 indicate that factor analysis is suitable, as the variables are strongly correlated.
* Values close to 0 indicate that factor analysis is unsuitable, as the variables are only weakly correlated.

In [ ]:
kmo_all, kmo_model = calculate_kmo(X_encoded)
print(f"\nKMO measure: {kmo_model:.3f} (0.6+ is often considered acceptable)")

## Principal Component Analysis, Factor Analysis, PCA

To solve the multicollinearity problem, we can use PCA to reduce the number of predictors. PCA transforms the original correlated variables into a smaller set of uncorrelated variables called principal components. These components can then be used in regression models to avoid multicollinearity issues.

### Application of PCA in Regression Problems:

* **Dimensionality Reduction:** PCA reduces the number of explanatory variables (features) by transforming the original variables into a smaller number of uncorrelated principal components. This can be particularly useful when dealing with many features, as fewer dimensions make the regression algorithm less prone to overfitting.
* **Reducing Multicollinearity:** In linear regression models, multicollinearity among independent variables can lead to unstable estimates. PCA helps eliminate this multicollinearity because the resulting principal components are orthogonal to each other.
* **Handling High-Dimensional Data:** For datasets with a large number of variables, PCA can be used to reduce the dimensions to a manageable level before starting the regression.
* **Reduced Overfitting Tendencies:** By removing redundant and highly correlated variables, PCA can help reduce the risk of overfitting by focusing the model on the most influential features.
* **Improved Model Performance:** In many cases, performing regression on the most important principal components instead of the original features can lead to better generalization and improved model performance on new, unseen data.
* **Interpretation of Feature Importance:** PCA provides insights into the importance of the original features through the variance explained by each principal component. This information can be used to identify which combinations of variables best represent the data.

### Loading Scores

* **Loading Scores** can be viewed as directional vectors in the feature space. The magnitude of the score indicates how dominant the variable is in the component, while the sign represents a direction.
* A **high positive loading score** means that the variable has a positive influence on the principal component and is correlated with it.
* A **high negative loading score** indicates that the variable is negatively correlated with the principal component, meaning it varies in the opposite direction.
* **Contribution to Variance:** The loading score values indicate how much each original variable contributes to the explained variance in the respective principal component.

## PCA for Car Sales

The Principal Component Analysis (PCA) is applied only to the features, not to the target variables.
Here we use the `pca_analysis` function to perform PCA on the dataset. The function takes the following parameters:

* `df`: The input DataFrame containing the features.
* `df_name`: The name of the DataFrame (for display purposes).
* `k`: The number of principal components to extract.
* `scaler`: The scaler to use for standardizing the data (e.g., `StandardScaler`).
* `max_scree`: The maximum number of components to display in the scree plot.


In [ ]:
top_features = pca_analysis(df=X_encoded, df_name="car_sales", k=10, scaler=StandardScaler(), max_scree=10)

In [ ]:
# Display the top features
print("Top Features from PCA:")
print(f"PCA 1: {top_features[0]}")
print(f"PCA 2: {top_features[1]}")

## PCA Application for Addressing Multicollinearity


### Determining the Number of Factors for Factor Analysis

The number of factors is set to `anz_fak=10` to reduce the dimensions. Factor analysis is performed with a Varimax rotation to improve the interpretability of the factors.

In [ ]:
anz_fak = 10
n_factors = min(anz_fak, X_encoded.shape[1])
fa = FactorAnalyzer(n_factors=n_factors, method="principal", rotation="varimax")
fa.fit(X_encoded)
# Factor loadings
factor_loadings = fa.loadings_
actual_factors = factor_loadings.shape[1]  # Number of factors actually extracted
print(f"actual_factors: {actual_factors}")
if actual_factors < n_factors:
    print(
        f"\nWarning: Only {actual_factors} factors could be extracted "
        f"(requested {n_factors})."
    )
factor_columns = [f"Factor{i+1}" for i in range(actual_factors)]

### Factor Loadings

Factor Loadings indicate how strongly each original variable is correlated with the extracted factors. High values suggest that the variable has a significant influence on the factor.

In [ ]:
# Print factor loadings with 2 decimals
print("Factor Loadings (rounded to 2 decimals):\n", np.round(factor_loadings, 2))

In [ ]:
# Create a DataFrame for the factor loadings
factor_loadings_df = pd.DataFrame(
    factor_loadings,
    index=X_encoded.columns,  # Original feature names
    columns=factor_columns  # Factor names
)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    factor_loadings_df,
    annot=True,  # Annotate with values
    fmt=".2f",  # Format values to 2 decimals
    cmap="coolwarm",  # Color map
    cbar=True  # Show color bar
)
plt.title("Factor Loadings Heatmap")
plt.xlabel("Factors")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

#### Factor Scores

The factor scores are the transformed values of the original variables based on the extracted factors. These scores can be used as new features in regression models.

In [ ]:
# Factor scores for each row (shape: [n_samples, actual_factors])
X_factor_scores = fa.transform(X_encoded)
print(f"X_factor_scores shape: {X_factor_scores.shape}")

# Adapt the factor column names to the actual factor count
df_factors = pd.DataFrame(X_factor_scores, columns=factor_columns)
print(f"df_factors shape: {df_factors.shape}")
print(f"df_factors head:\n{df_factors.head()}")

### Creating the Regression Model with all ten Extracted Factors

In [ ]:
X_model = sm.add_constant(df_factors)
model_factors = sm.OLS(y, X_model).fit()
print("\nRegression on Factor Scores:")
print(model_factors.summary())

In [ ]:
coeffs_table = compute_coefficients_table(
    model=model_factors,
    X_encoded=X_model,
    y=y,
    vif_table=None
)
print("\nCoefficients Table:")
print(coeffs_table)

As expected, the collinearity statistics show that the factor values are uncorrelated. Additionally, it is important to note that the variability of the coefficient estimates in this model is not artificially inflated by collinearity. Consequently, the coefficient estimates are larger relative to their standard errors compared to the original model. This means that more factors are identified as statistically significant, which can influence your final results if you aim to create a model that includes only significant effects.

To verify, we calculate the condition indices for the extracted factors. These should all be close to 1, indicating that there are no serious issues with multicollinearity.

In [ ]:
X_cond = copy.deepcopy(df_factors)
condition_index_df = condition_index(X_cond)
print("\nCondition Index:")
print(condition_index_df)

### Comparison of Model Performance

In [ ]:
# Predictions from the Linear Regression Model (model)
predictions_linear = model.predict(X_encoded_with_const)

# Predictions from the Factor Analysis Regression Model (model_factors)
predictions_factors = model_factors.predict(X_model)

# Calculate R-squared and Adjusted R-squared for both models
r2_linear = model.rsquared
adj_r2_linear = model.rsquared_adj

r2_factors = model_factors.rsquared
adj_r2_factors = model_factors.rsquared_adj

# Calculate Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) for both models
mse_linear = mean_squared_error(y, predictions_linear)
rmse_linear = np.sqrt(mse_linear)

mse_factors = mean_squared_error(y, predictions_factors)
rmse_factors = np.sqrt(mse_factors)

# Print the comparison
print("Vergleich der beiden Modelle")
print("\nLinear Regression Model:")
print(f"R-squared: {r2_linear:.4f}")
print(f"Adjusted R-squared: {adj_r2_linear:.4f}")
print(f"MSE: {mse_linear:.4f}")
print(f"RMSE: {rmse_linear:.4f}")

print("\nFactor Analysis Regression Model:")
print(f"R-squared: {r2_factors:.4f}")
print(f"Adjusted R-squared: {adj_r2_factors:.4f}")
print(f"MSE: {mse_factors:.4f}")
print(f"RMSE: {rmse_factors:.4f}")

If the R-squared and Adjusted R-squared values for `model_factors` are close to those of `model`, this indicates that the regression model based on factor analysis performs similarly well while potentially reducing multicollinearity. Lower MSE and RMSE values suggest better predictive performance.


### Creating the Regression Model with three Extracted Factors only

In [ ]:
# Create a regression model using only the first three factors
# select the first three factors
df_factors = df_factors.iloc[:, :3]

In [ ]:
X_model = sm.add_constant(df_factors)
model_factors = sm.OLS(y, X_model).fit()
print("\nRegression on Factor Scores:")
print(model_factors.summary())

In [ ]:
coeffs_table = compute_coefficients_table(
    model=model_factors,
    X_encoded=X_model,
    y=y,
    vif_table=None
)
print("\nCoefficients Table:")
print(coeffs_table)

As expected, the collinearity statistics show that the factor values are uncorrelated. Additionally, it is important to note that the variability of the coefficient estimates in this model is not artificially inflated by collinearity. Consequently, the coefficient estimates are larger relative to their standard errors compared to the original model. This means that more factors are identified as statistically significant, which can influence your final results if you aim to create a model that includes only significant effects.

To verify, we calculate the condition indices for the extracted factors. These should all be close to 1, indicating that there are no serious issues with multicollinearity.

In [ ]:
X_cond = copy.deepcopy(df_factors)
condition_index_df = condition_index(X_cond)
print("\nCondition Index:")
print(condition_index_df)

### Comparison of Model Performance of the Reduced Model and the Full Model

In [ ]:
# Predictions from the Linear Regression Model (model)
predictions_linear = model.predict(X_encoded_with_const)

# Predictions from the Factor Analysis Regression Model (model_factors)
predictions_factors = model_factors.predict(X_model)

# Calculate R-squared and Adjusted R-squared for both models
r2_linear = model.rsquared
adj_r2_linear = model.rsquared_adj

r2_factors = model_factors.rsquared
adj_r2_factors = model_factors.rsquared_adj

# Calculate Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) for both models
mse_linear = mean_squared_error(y, predictions_linear)
rmse_linear = np.sqrt(mse_linear)

mse_factors = mean_squared_error(y, predictions_factors)
rmse_factors = np.sqrt(mse_factors)

# Print the comparison
print("Vergleich der beiden Modelle")
print("\nLinear Regression Model:")
print(f"R-squared: {r2_linear:.4f}")
print(f"Adjusted R-squared: {adj_r2_linear:.4f}")
print(f"MSE: {mse_linear:.4f}")
print(f"RMSE: {rmse_linear:.4f}")

print("\nFactor Analysis Regression Model:")
print(f"R-squared: {r2_factors:.4f}")
print(f"Adjusted R-squared: {adj_r2_factors:.4f}")
print(f"MSE: {mse_factors:.4f}")
print(f"RMSE: {rmse_factors:.4f}")

If the R-squared and Adjusted R-squared values for `model_factors` are close to those of `model`, this indicates that the regression model based on factor analysis performs similarly well while potentially reducing multicollinearity. Lower MSE and RMSE values suggest better predictive performance.



## Summary

### Interpretation of the Regression Model `model_factors`

The `model_factors` regression model is based on factor analysis, which reduces the dimensionality of the input data by transforming the original variables into a smaller group of uncorrelated factors. These factors are linear combinations of the original variables and are designed to capture the underlying structure of the data.

**Factors:** Instead of using the original variables, the regression is performed on the extracted factors. Each factor represents a weighted combination of the original variables.

**Coefficients:** The coefficients in the `model_factors` regression model indicate the relationship between the target variable (`y`) and the extracted factors.

### Differences Compared to the Standard OLS Model

**Input Variables:**

- **OLS Model (`model`):** Uses the original variables (`X_encoded`) as predictors.
- **Factor Analysis Model (`model_factors`):** Uses the extracted factors (`df_factors`) as predictors.

**Multicollinearity:**

- **OLS Model:** Can suffer from multicollinearity if the predictors are highly correlated, leading to unstable coefficients and inflated standard errors.
- **Factor Analysis Model:** Reduces multicollinearity by using uncorrelated factors as predictors.

**Interpretability:**

- **OLS Model:** Coefficients correspond directly to the original variables, making it easier to interpret the influence of each variable on the target variable.
- **Factor Analysis Model:** Coefficients relate to abstract factors, which are combinations of the original variables, making interpretation more challenging.

**Dimensionality:**

- **OLS Model:** Uses all original variables, which may include redundant or irrelevant features.
- **Factor Analysis Model:** Reduces the number of predictors by combining the original variables into a smaller number of factors.

### Advantages of Using `model_factors`

- **Reduced Multicollinearity:** By using uncorrelated factors, the model avoids instability caused by multicollinearity.
- **Dimensionality Reduction:** The model uses fewer predictors, improving computational efficiency and generalization.
- **Focus on Underlying Structure:** Factor analysis captures the latent structure of the data, providing better insights into the relationships between variables.

### Disadvantages of Using `model_factors`

- **Loss of Interpretability:** Factors are abstract combinations of the original variables, making it harder to directly interpret the coefficients. To understand the influence of individual variables on the target variable, factor loadings must be analyzed.
- **Potential Information Loss:** If too few factors are retained, information from the original variables may be lost, reducing predictive accuracy.
- **Complexity:** The process of extracting factors and interpreting their meaning adds complexity to the modeling process.
- **Dependence on Factor Selection:** The number of factors to retain is subjective and can affect model performance. Too few factors may oversimplify the data, while too many factors may reintroduce multicollinearity.

### Summary

- `model_factors` is a good choice when multicollinearity is a problem and the focus is on predictive performance rather than interpretability.
- The `model` is preferable when interpretability is crucial and multicollinearity is not a significant issue.

## Using `model_factors` in Other Models

### Random Forest Regressor with the Full Dataset

* First, use the original variables.

In [ ]:
# ------------------------------------------------------------------------
# 1. Prepare Data
# ------------------------------------------------------------------------

# Use the original input features (X_encoded) as predictors
X_original = X_encoded

# Split the data into training and testing sets
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_original, y, test_size=0.2, random_state=42)

# ------------------------------------------------------------------------
# 2. Fit Random Forest Model
# ------------------------------------------------------------------------

# Initialize the Random Forest Regressor
rf_model_orig = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model_orig.fit(X_train_orig, y_train_orig)

# ------------------------------------------------------------------------
# 3. Evaluate the Model
# ------------------------------------------------------------------------

# Make predictions on the test set
y_pred_orig = rf_model_orig.predict(X_test_orig)

# Calculate evaluation metrics
r2_rf_orig = r2_score(y_test_orig, y_pred_orig)
mse_rf_orig = mean_squared_error(y_test_orig, y_pred_orig)
rmse_rf_orig = np.sqrt(mse_rf_orig)

# Print the results
print("\nRandom Forest Model (using original data):")
print(f"R-squared: {r2_rf_orig:.4f}")
print(f"MSE: {mse_rf_orig:.4f}")
print(f"RMSE: {rmse_rf_orig:.4f}")

### Random Forest Regressor with Extracted Factors

* Then use the extracted factors.

In [ ]:
# ------------------------------------------------------------------------
# 1. Prepare Data
# ------------------------------------------------------------------------

# Use the extracted factors as predictors
X_factors = df_factors


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_factors, y, test_size=0.2, random_state=42)

# ------------------------------------------------------------------------
# 2. Fit Random Forest Model
# ------------------------------------------------------------------------

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# ------------------------------------------------------------------------
# 3. Evaluate the Model
# ------------------------------------------------------------------------

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Calculate evaluation metrics
r2_rf = r2_score(y_test, y_pred)
mse_rf = mean_squared_error(y_test, y_pred)
rmse_rf = np.sqrt(mse_rf)

# Print the results
print("\nRandom Forest Model (using extracted factors):")
print(f"R-squared: {r2_rf:.4f}")
print(f"MSE: {mse_rf:.4f}")
print(f"RMSE: {rmse_rf:.4f}")

### Comparison of the two Random Forest Models {#sec-rf-comparison}

In [ ]:
# Print comparison of Random Forest models
print("\nComparison of Random Forest Models:")
print("\nUsing Extracted Factors:")
print(f"R-squared: {r2_rf:.4f}")
print(f"MSE: {mse_rf:.4f}")
print(f"RMSE: {rmse_rf:.4f}")

print("\nUsing Original Data:")
print(f"R-squared: {r2_rf_orig:.4f}")
print(f"MSE: {mse_rf_orig:.4f}")
print(f"RMSE: {rmse_rf_orig:.4f}")

## Multicollinearity: Conclusion and Recommendation

* PCA is a standard method for addressing multicollinearity.
* The principal components determined using linear regression are no longer multicollinear.
* These components can also be used for other models, such as Random Forest.
* The principal components are not easy to interpret.


## Videos: Principal Component Analysis (PCA)

* Video: [Principal Component Analysis (PCA), Step-by-Step](https://youtu.be/FgakZw6K1QQ?si=lmXhc-bpOqb7RmDP)
* Video: [PCA - Practical Tips](https://youtu.be/oRvgq966yZg?si=TIUsxNItfyYOjTLt)
* Video: [PCA in Python](https://youtu.be/Lsue2gEM9D0?si=_fV_RzK8j1jwcb-e)


## Jupyter Notebook

:::{.callout-note}

* The Jupyter-Notebook of this lecture is available on GitHub in the [Hyperparameter-Tuning-Cookbook Repository](https://github.com/sequential-parameter-optimization/Hyperparameter-Tuning-Cookbook/blob/main/100_ddmo_pca.ipynb)

:::
